In [1]:
import torch
import torchvision

from resnet import resnet as score_model
from retinanet import model as detection_model

In [2]:
BATCH_SIZE = 1

SCORE_MODEL_PATH = './data/score_state_dict.pth'
DET_MODEL_PATH = './data/detection_state_dict.pth'

OPSET_VERSION = 11 # 2020.08 - 10 for openvino, 11 for trt

## score net

In [ ]:
# score net

IMAGE_SIZE = 224

dummy_input = torch.randn(BATCH_SIZE, 3, IMAGE_SIZE, IMAGE_SIZE, device='cuda')

# dynamic batch size
# for vino, we must convert model with -b (this only works for cpu/gpu)
'''
input_names = ['input_1']
output_names = ['output_1']

dynamic_axes = {input_names[0]: {0: 'batch'}}

for output_name in output_names:
    dynamic_axes[output_name] = dynamic_axes[input_names[0]]
'''

# load network
score_net = score_model.resnet50(num_classes=1).cuda()
score_net.load_state_dict(torch.load(SCORE_MODEL_PATH))

torch.onnx.export(
    score_net,
    dummy_input,
    'score_net.onnx',
    verbose=True,
    opset_version=OPSET_VERSION,
    # input_names=input_names,
    # output_names=output_names,
    # dynamic_axes=dynamic_axes
)

## det net

In [3]:
# upsample fix

import torch.onnx.symbolic_opset11 as onnx_symbolic

def upsample_nearest2d(g, input, output_size, *args):
    # Currently, TRT 7.1 ONNX Parser does not support all ONNX ops
    # needed to support dynamic upsampling ONNX forumlation
    # Here we hardcode scale=2 as a temporary workaround
    scales = g.op("Constant", value_t=torch.tensor([1., 1., 2., 2.]))
    empty_tensor = g.op("Constant", value_t=torch.tensor([], dtype=torch.float32))
    return g.op("Resize", input, empty_tensor, scales, mode_s="nearest", nearest_mode_s="floor")

onnx_symbolic.upsample_nearest2d = upsample_nearest2d

In [ ]:
# det net

IMAGE_SIZE = 512

dummy_input = torch.randn(BATCH_SIZE, 3, IMAGE_SIZE, IMAGE_SIZE, device='cuda')

det_net = detection_model.resnet50(num_classes=5).cuda()
det_net.load_state_dict(torch.load(DET_MODEL_PATH))

# torch.onnx.export(det_net, dummy_input, 'det_net.onnx', verbose=True)
# for now, intel openvino only supports opset-10 resizing
det_net.export_onnx(dummy_input, 'det_net.onnx', op_set_version=OPSET_VERSION)